**Mounting Drive to access Dataset**

**Importing Relevant Libraries**

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.utils import np_utils
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input, Conv2D, LeakyReLU, MaxPooling2D, concatenate, LSTM, Reshape, Dense
from keras.callbacks import EarlyStopping
from tensorflow import keras
import tensorflow as tf
from sklearn import svm
from sklearn.linear_model import SGDClassifier

import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.metrics import classification_report




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Pre_processing Functions

In [ ]:
def get_x_and_y(data): #function to separate data and labels, we studied the code referenced in the githubs on the paper to understand the pre-processing pipeline and then wrote our own code with changes in it as we thought that the referenced code had a few mistakes.
  df1 = data[:40, :]
  new_df1=np.transpose(df1)
  df2= data[-5:,:]
  new_df2= np.transpose(df2)


  return  np.array(new_df1), np.array(new_df2)



In [ ]:
def prepare_x(x,Time_frame): # function to prepare data according to timeframe

  N,Price_x= x.shape[0],x.shape[1]
  empty_arr= np.zeros([(N-Time_frame+1), Time_frame, Price_x])
  for j in range(Time_frame,N+1):
    empty_arr[j-Time_frame] = x[j-Time_frame:j, :]
  empty_arr = empty_arr.reshape(empty_arr.shape + (1,))
  return empty_arr






In [ ]:
def prepare_y(y,x,prediction_horizon ,time_frame): # fucntion to create labels for time_frame
  N= x.shape[0]
  y_new= y[time_frame - 1:N]
  y_new = y_new[:,prediction_horizon] - 1
  y_new = np_utils.to_categorical(y_new, 3)
  return y_new

In [ ]:
def normalize_SVMoutput(labels): #function to normalize SVM inputs
  empty_arr= np.zeros(labels.shape[0])
  for i in range(len(labels)):
    empty_arr[i]= labels[i]-1
  return empty_arr

Splitting the test data into training set and validation set. 80% will be training while 20% will be validation.

In [ ]:
train_1= np.loadtxt("/content/drive/MyDrive/CS4701/Datasets/published/BenchmarkDatasets/BenchmarkDatasets/NoAuction/3.NoAuction_DecPre/NoAuction_DecPre_Testing/Test_Dst_NoAuction_DecPre_CF_2.txt")

In [ ]:
train_1_final = train_1[:, :int(np.floor(train_1.shape[1] * 0.8))]
train_1_val = train_1[:, int(np.floor(train_1.shape[1] * 0.8)):]


In [ ]:
validate_x,validate_y=get_x_and_y(train_1_val)

In [ ]:
train_2=np.loadtxt("/content/drive/MyDrive/CS4701/Datasets/published/BenchmarkDatasets/BenchmarkDatasets/NoAuction/3.NoAuction_DecPre/NoAuction_DecPre_Training/Train_Dst_NoAuction_DecPre_CF_4.txt")

In [ ]:
train_2_final = train_2[:, :int(np.floor(train_2.shape[1] * 0.8))]
test_2 = train_2[:, int(np.floor(train_2.shape[1] * 0.8)):]

In [ ]:
train_2_x,train_2_y= get_x_and_y(train_2_final)

In [ ]:
train_2_xfinal= prepare_x(train_2_x,100)

In [ ]:
train_2_yfinal= prepare_y(train_2_y,train_2_x,4,100)

In [ ]:
test_2_x,test_2_y= get_x_and_y(test_2)

In [ ]:
test_2_xfinal= prepare_x(test_2_x,100)

In [ ]:
test_2_yfinal= prepare_y(test_2_y,test_2_x,4,100)

In [ ]:
train_1_x,train_1_y= get_x_and_y(train_1_final)

In [ ]:
final_x_train,final_x_val= prepare_x(train_1_x,100),prepare_x(validate_x,100)

In [ ]:
final_y_train,final_y_val=prepare_y(train_1_y,train_1_x,3,100),prepare_y(validate_y,validate_x,3,100)

In [ ]:
print(train_2_x.shape)

(114773, 40)


In [ ]:
print(test_2_xfinal.shape)

(28595, 100, 40, 1)


In [ ]:
print(np.array(final_x_train).shape)
print(final_x_val.shape)

(22729, 100, 40, 1)
(5608, 100, 40, 1)


In [ ]:
print(np.array(test_2_yfinal).shape)
print(train_2_yfinal.shape)

(28595, 3)
(114674, 3)


In [ ]:
testing_data = np.loadtxt("/content/drive/MyDrive/CS4701/Datasets/published/BenchmarkDatasets/BenchmarkDatasets/NoAuction/3.NoAuction_DecPre/NoAuction_DecPre_Testing/Test_Dst_NoAuction_DecPre_CF_2.txt")

In [ ]:
testing_x,testing_y= get_x_and_y(testing_data)

In [ ]:
testing_final_x= prepare_x(testing_x,100)

In [ ]:
testing_final_y= prepare_y(testing_y,testing_x,4,100)

In [ ]:
svm_train2= train_2_yfinal.flatten()

In [ ]:
svm_train1= train_2_xfinal.flatten()

In [ ]:
print(train_2_y.shape)

(114773, 5)


Baseline Model using Supervised machine learning

In [ ]:
svm = SGDClassifier(loss='hinge')

# Fit the model
svm.fit(train_2_x,train_2_y[:,4])


SGDClassifier()

In [ ]:
svm_prediction = svm.predict(testing_x) # predicting labels

In [ ]:
new_arr1,new_arr2= normalize_SVMoutput(svm_prediction),normalize_SVMoutput(testing_y[:,4]) # Comparing SVM prediction using Prediction Horizon 4

In [ ]:
new_arr3,new_arr4= normalize_SVMoutput(svm_prediction),normalize_SVMoutput(testing_y[:,3]) # Comparing SVM prediction using Prediction Horizon 3

In [ ]:
print(classification_report(new_arr2,new_arr1))
print(accuracy_score(new_arr2,new_arr1))

              precision    recall  f1-score   support

         0.0       0.39      0.56      0.46     10931
         1.0       0.23      0.03      0.05      6462
         2.0       0.45      0.48      0.46     11142

    accuracy                           0.41     28535
   macro avg       0.36      0.36      0.33     28535
weighted avg       0.38      0.41      0.37     28535

0.4100928684072192


In [ ]:
print(classification_report(new_arr4,new_arr3))
print(accuracy_score(new_arr4,new_arr3))

              precision    recall  f1-score   support

         0.0       0.32      0.56      0.41      9091
         1.0       0.35      0.03      0.05     10211
         2.0       0.38      0.49      0.43      9233

    accuracy                           0.35     28535
   macro avg       0.35      0.36      0.30     28535
weighted avg       0.35      0.35      0.29     28535

0.34659190467846507


CNN-LSTM Model Parameters and Architecture using Paper

In [ ]:
filter_size= 16 #for convolution
alpha_no = 0.01
lstm_size= 64
inception_size= 32
kernel_size_1=(1,2)
kernel_size_2=(4,1)
kernel_size_3= (1,10)
kernel_size_4= (1,1)
kernel_size_5= (3,1)
kernel_size_6=(5,1)


In [ ]:
def cnn_model_2(time_frame, entries, lstm_size,inception_size,filter_size,alpha):
  tensor= Input(shape= (time_frame,entries,1))

  #convolution according to paper details
  layer_1= Conv2D(filter_size,kernel_size_1,strides=(1,2))(tensor)
  layer_1= LeakyReLU(alpha= alpha_no)(layer_1)
  layer_1 = Conv2D(filter_size,kernel_size_2 , padding='same')(layer_1)
  layer_1 = LeakyReLU(alpha=alpha_no)(layer_1)
  layer_1 = Conv2D(filter_size, kernel_size_2, padding='same')(layer_1)
  layer_1 = LeakyReLU(alpha=alpha_no)(layer_1)

  layer_2 = Conv2D(filter_size, kernel_size_1, strides=(1, 2))(layer_1)
  layer_2 = LeakyReLU(alpha=alpha_no)(layer_2)
  layer_2 = Conv2D(filter_size,kernel_size_2, padding='same')(layer_2)
  layer_2 = LeakyReLU(alpha=alpha_no)(layer_2)
  layer_2 = Conv2D(filter_size, kernel_size_2, padding='same')(layer_2)
  layer_2 = LeakyReLU(alpha=alpha_no)(layer_2)

  layer_3 = Conv2D(filter_size,kernel_size_3 )(layer_2)
  layer_3 = LeakyReLU(alpha=alpha_no)(layer_3)
  layer_3 = Conv2D(filter_size, kernel_size_2, padding='same')(layer_3)
  layer_3 = LeakyReLU(alpha=alpha_no)(layer_3)
  layer_3 = Conv2D(filter_size, kernel_size_2, padding='same')(layer_3)
  layer_3 = LeakyReLU(alpha=alpha_no)(layer_3)

  inception_1 = Conv2D(inception_size, kernel_size_4, padding='same')(layer_3)
  inception_1 = LeakyReLU(alpha=alpha_no)(inception_1)
  inception_1 = Conv2D(inception_size, kernel_size_5, padding='same')(inception_1)
  inception_1 = LeakyReLU(alpha=alpha_no)(inception_1)

  inception_2 = Conv2D(inception_size, kernel_size_4, padding='same')(layer_3)
  inception_2 = LeakyReLU(alpha=alpha_no)(inception_2)
  inception_2 = Conv2D(inception_size, kernel_size_6, padding='same')(inception_2)
  inception_2 = LeakyReLU(alpha=alpha_no)(inception_2)

  inception_3 = MaxPooling2D(kernel_size_5, strides=(1,1), padding='same')(layer_3)
  inception_3 = Conv2D(inception_size, kernel_size_4, padding='same')(inception_3)
  inception_3 = LeakyReLU(alpha=alpha_no)(inception_3)

  combined_layers= keras.layers.concatenate([inception_1, inception_2, inception_3], axis=3)
  combined_final = keras.layers.Reshape((combined_layers.shape[1], combined_layers.shape[3]))(combined_layers)
  combined_final = keras.layers.Dropout(0.2, noise_shape=(None, 1, int(combined_final.shape[2])))(combined_final, training=True)

  lstm = keras.layers.LSTM(lstm_size)(combined_final)

  out = keras.layers.Dense(3, activation='softmax')(lstm)
  model = Model(inputs=tensor, outputs=out)
  opt = keras.optimizers.Adam(lr=0.0001)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model









In [ ]:
def cnn_model_2_no_dropout(time_frame, entries, lstm_size,inception_size,filter_size,alpha):
  tensor= Input(shape= (time_frame,entries,1))

  #convolution according to paper details
  layer_1= Conv2D(filter_size,kernel_size_1,strides=(1,2))(tensor)
  layer_1= LeakyReLU(alpha= alpha_no)(layer_1)
  layer_1 = Conv2D(filter_size,kernel_size_2 , padding='same')(layer_1)
  layer_1 = LeakyReLU(alpha=alpha_no)(layer_1)
  layer_1 = Conv2D(filter_size, kernel_size_2, padding='same')(layer_1)
  layer_1 = LeakyReLU(alpha=alpha_no)(layer_1)

  layer_2 = Conv2D(filter_size, kernel_size_1, strides=(1, 2))(layer_1)
  layer_2 = LeakyReLU(alpha=alpha_no)(layer_2)
  layer_2 = Conv2D(filter_size,kernel_size_2, padding='same')(layer_2)
  layer_2 = LeakyReLU(alpha=alpha_no)(layer_2)
  layer_2 = Conv2D(filter_size, kernel_size_2, padding='same')(layer_2)
  layer_2 = LeakyReLU(alpha=alpha_no)(layer_2)

  layer_3 = Conv2D(filter_size,kernel_size_3 )(layer_2)
  layer_3 = LeakyReLU(alpha=alpha_no)(layer_3)
  layer_3 = Conv2D(filter_size, kernel_size_2, padding='same')(layer_3)
  layer_3 = LeakyReLU(alpha=alpha_no)(layer_3)
  layer_3 = Conv2D(filter_size, kernel_size_2, padding='same')(layer_3)
  layer_3 = LeakyReLU(alpha=alpha_no)(layer_3)

  inception_1 = Conv2D(inception_size, kernel_size_4, padding='same')(layer_3)
  inception_1 = LeakyReLU(alpha=alpha_no)(inception_1)
  inception_1 = Conv2D(inception_size, kernel_size_5, padding='same')(inception_1)
  inception_1 = LeakyReLU(alpha=alpha_no)(inception_1)

  inception_2 = Conv2D(inception_size, kernel_size_4, padding='same')(layer_3)
  inception_2 = LeakyReLU(alpha=alpha_no)(inception_2)
  inception_2 = Conv2D(inception_size, kernel_size_6, padding='same')(inception_2)
  inception_2 = LeakyReLU(alpha=alpha_no)(inception_2)

  inception_3 = MaxPooling2D(kernel_size_5, strides=(1,1), padding='same')(layer_3)
  inception_3 = Conv2D(inception_size, kernel_size_4, padding='same')(inception_3)
  inception_3 = LeakyReLU(alpha=alpha_no)(inception_3)

  combined_layers= keras.layers.concatenate([inception_1, inception_2, inception_3], axis=3)
  combined_final = keras.layers.Reshape((combined_layers.shape[1], combined_layers.shape[3]))(combined_layers)

  lstm = keras.layers.LSTM(lstm_size)(combined_final)

  out = keras.layers.Dense(3, activation='softmax')(lstm)
  model = Model(inputs=tensor, outputs=out)
  opt = keras.optimizers.Adam(lr=0.0001)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model


In [ ]:
model_new= cnn_model_2(100,40, lstm_size,inception_size, filter_size,alpha_no)

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model_nodropout= cnn_model_2_no_dropout(100,40, lstm_size,inception_size, filter_size,alpha_no)

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model_new.summary()

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="/content/drive/MyDrive/CS4701/checkpoints/",
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)

model_new.fit(train_2_xfinal, train_2_yfinal, validation_data=(test_2_xfinal,test_2_yfinal),
            epochs=100, batch_size=32, verbose=2, callbacks=[model_checkpoint_callback])

Training Model

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="/content/drive/MyDrive/CS4701/checkpoints",
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)

model_nodropout.fit(train_2_xfinal, train_2_yfinal, validation_data=(test_2_xfinal,test_2_yfinal),
            epochs=100, batch_size=32, verbose=2, callbacks=[model_checkpoint_callback])

Epoch 1/100
3584/3584 - 502s - loss: 1.0280 - accuracy: 0.4218 - val_loss: 1.0968 - val_accuracy: 0.3736 - 502s/epoch - 140ms/step
Epoch 2/100
3584/3584 - 519s - loss: 0.9858 - accuracy: 0.4386 - val_loss: 1.0832 - val_accuracy: 0.3761 - 519s/epoch - 145ms/step
Epoch 3/100
3584/3584 - 521s - loss: 0.9131 - accuracy: 0.4758 - val_loss: 1.0182 - val_accuracy: 0.4863 - 521s/epoch - 145ms/step
Epoch 4/100
3584/3584 - 518s - loss: 0.8909 - accuracy: 0.4882 - val_loss: 0.9643 - val_accuracy: 0.4901 - 518s/epoch - 145ms/step
Epoch 5/100
3584/3584 - 521s - loss: 0.8810 - accuracy: 0.4969 - val_loss: 1.0726 - val_accuracy: 0.4293 - 521s/epoch - 145ms/step
Epoch 6/100
3584/3584 - 518s - loss: 0.8713 - accuracy: 0.5026 - val_loss: 0.9287 - val_accuracy: 0.4985 - 518s/epoch - 144ms/step
Epoch 7/100
3584/3584 - 519s - loss: 0.8621 - accuracy: 0.5160 - val_loss: 0.9429 - val_accuracy: 0.4800 - 519s/epoch - 145ms/step
Epoch 8/100
3584/3584 - 517s - loss: 0.8503 - accuracy: 0.5436 - val_loss: 0.9375 -

Testing Dataset Pre-process

In [ ]:
test_1= np.loadtxt("/content/drive/MyDrive/CS4701/Datasets/published/BenchmarkDatasets/BenchmarkDatasets/NoAuction/3.NoAuction_DecPre/NoAuction_DecPre_Testing/Test_Dst_NoAuction_DecPre_CF_5.txt")

In [ ]:
test_1_x, test_1_y= get_x_and_y(test_1)

In [ ]:
test_1_xfinal= prepare_x(test_1_x,100)

In [ ]:
test_1_yfinal= prepare_y(test_1_y,test_1_x,3,100)

Model Evaluation functions

In [ ]:
a= model_new.load_weights("/content/drive/MyDrive/CS4701/checkpoints")

In [ ]:
pred2= model_nodropout.predict(testing_final_x)

889/889 [==============================] - 65s 72ms/step


In [ ]:
pred = model_new.predict(testing_final_x)

889/889 [==============================] - 44s 48ms/step


In [ ]:
print('accuracy_score:', accuracy_score(np.argmax(testing_final_y, axis=1), np.argmax(pred, axis=1)))
print(classification_report(np.argmax(testing_final_y, axis=1), np.argmax(pred, axis=1), digits=4)) # source for this code is listed in the paper references

accuracy_score: 0.7054086369390913
              precision    recall  f1-score   support

           0     0.6828    0.7470    0.7135     10906
           1     0.8301    0.5732    0.6781      6434
           2     0.6818    0.7412    0.7103     11096

    accuracy                         0.7054     28436
   macro avg     0.7316    0.6871    0.7006     28436
weighted avg     0.7158    0.7054    0.7042     28436



In [ ]:
print('accuracy_score:', accuracy_score(np.argmax(testing_final_y, axis=1), np.argmax(pred2, axis=1)))
print(classification_report(np.argmax(testing_final_y, axis=1), np.argmax(pred2, axis=1), digits=4))